<a href="https://colab.research.google.com/github/kawata-yuya/Hokkaido-Coronavirus-Positive-Regression-AI/blob/master/rinna_japanese_gpt_1b_chat_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 必要なモジュールのインストール。
# Huggingface Transformersのインストール
!pip install transformers==4.15.0

# Sentencepieceのインストール
!pip install sentencepiece==0.1.96

# Huggingface Datasetsのインストール
!pip install datasets==1.18.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.4 MB 24.6 MB/s 
     |████████████████████████████████| 3.3 MB 47.8 MB/s 
     |████████████████████████████████| 596 kB 67.6 MB/s 
     |████████████████████████████████| 880 kB 50.2 MB/s 
     |████████████████████████████████| 101 kB 15.2 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=5b18be0262dad37d87f79c9b71d9ac9ca4fce2af9c412974b93b6c8d41da8f0d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 27.9 MB/s 
Looking 

In [ ]:
#@title 言語モデル rinna/japanese-gpt-1bを読み込み
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")

if torch.cuda.is_available():
    model = model.to("cuda")

Downloading:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/153 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/283 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

In [ ]:
#@title チャットの定義

class TalkChatGPT:
    def __init__(self, title:str, username:str, partnername = 'りんな') -> None:
        self.title = title
        self.username = username
        self.partnername = partnername
        self.text = f"{self.title} "
        self.lastres = ''

    def gen_respons(self):
        self.text += f'{self.partnername}:<'
        token_ids = tokenizer.encode(self.text, add_special_tokens=False, return_tensors="pt")

        length = len(self.text) + 30

        with torch.no_grad():
            output_ids = model.generate(
                token_ids.to(model.device),
                max_length=length,
                min_length=length,
                do_sample=True,
                top_k=500,
                top_p=0.95,
                pad_token_id=tokenizer.pad_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bad_word_ids=[[tokenizer.unk_token_id]]
            )

        output = tokenizer.decode(output_ids.tolist()[0])
        # print(output)
        self.lastres = output[len(self.text):].split('>')[0]
        
        self.text += f'{self.lastres}> '

        return self.lastres
    
    def talk(self, user_text_data):
        self.text += f'{self.username}:<{user_text_data}> '
        return self.gen_respons()
    
    def replace(self, new_partner_text):
        self.text = self.text[:len(self.lastres)-2] + new_partner_text + '> '
        return
    
    def regen_respons(self):
        self.text = self.text[:-len(self.lastres)-2]
        token_ids = tokenizer.encode(self.text, add_special_tokens=False, return_tensors="pt")

        length = len(self.text) + 30

        with torch.no_grad():
            output_ids = model.generate(
                token_ids.to(model.device),
                max_length=length,
                min_length=length,
                do_sample=True,
                top_k=500,
                top_p=0.95,
                pad_token_id=tokenizer.pad_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
                bad_word_ids=[[tokenizer.unk_token_id]]
            )

        output = tokenizer.decode(output_ids.tolist()[0])
        # print(output)
        self.lastres = output[len(self.text):].split('>')[0]
        
        self.text += f'{self.lastres}> '

        return self.lastres
    
    def reset(self):
        self.text= f"{self.title} "
        


In [ ]:
#@title チャットの開始

# ユーザーの名前
username = '\u305F\u308D\u3046'         #@param{type:"string"}
# 相手(AI)の名前
partnername = '\u306F\u306A\u3053'      #@param{type:"string"}
# 会話の詳細。それぞれの職業とか性格とか。特に書式はない。
chat_details = '\u305F\u308D\u3046\u3068\u3001\u306F\u306A\u3053\u306E\u5B66\u6821\u3067\u306E\u4F1A\u8A71\u3002 \u305F\u308D\u3046\u3068\u3001\u306F\u306A\u3053\u306F12\u6B73\u306E\u5C0F\u5B66\u751F\u3002'                       #@param{type:"string"}
tcg = TalkChatGPT(chat_details, username, partnername)

while True:
    user_text = input(f'{username}: ')

    if user_text == '.quit':
        print(f'{username}と{partnername}の会話を終了します。')
        break

    elif user_text == '.reset':
        tcg.reset()
        continue

    elif user_text == '.replace':
        replace_text = input('replace_text:')
        tcg.replace(replace_text)
        print(f'{partnername}(AI):{replace_text}')
        continue

    elif user_text == '.regen':
        print(f'{partnername}(AI):', tcg.regen_respons())
        continue

    print(f'{partnername}(AI):', tcg.talk(user_text))

たろう: 昨日のピアノのレッスンはどうだった?
はなこ(AI): 昨日は良くて。 綺麗に弾けまして
たろう: なんの曲を引いたの?
はなこ(AI): バッハのインベンションを練習した
たろう: そうなんだー。将来ピアニストになりたいの?
はなこ(AI): はい
たろう: 週に何時間ピアノの練習をしてるの。
はなこ(AI): 平均すると8時間くらい練習しています
たろう: すごいね!!ピアノを始めようとしたきっかけは?
はなこ(AI): お兄ちゃんのピアノを見たとき、すてきだな。私も弾きたいなーって
たろう: そうなんだね!!
はなこ(AI): お兄ちゃんみたいに弾けるようになりたいな
たろう: きっと、はなこちゃんもなれるさ!!
はなこ(AI): がんばります
たろう: がんばってね。
はなこ(AI): ありがとうございます!
たろう: .quit
たろうとはなこの会話を終了します。
